In [ ]:
import matplotlib.pyplot as plt
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
# define input size, hidden layer size, output size
D_i, D_k, D_o = 10, 40, 5
# create model with two hidden layers
model = nn.Sequential(
 nn.Linear(D_i, D_k),
 nn.ReLU(),
 nn.Linear(D_k, D_k),
 nn.ReLU(),
 nn.Linear(D_k, D_o))

He initialization of weights

In [ ]:
def weights_init(layer_in):
 if isinstance(layer_in, nn.Linear):
  nn.init.kaiming_normal_(layer_in.weight)
  layer_in.bias.data.fill_(0.0)
model.apply(weights_init)

choose least squares loss function

In [ ]:
criterion = nn.MSELoss()
# construct SGD optimizer and initialize learning rate and momentum
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1, momentum=0.9)
# object that decreases learning rate by half every 10 epochs
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
# create 100 random data points and store in data loader class
x = torch.randn(100, D_i)
y = torch.randn(100, D_o)
data_loader = DataLoader(TensorDataset(x,y), batch_size=10, shuffle=True)

for plotting data

In [ ]:
fig, ax = plt.subplots()
epochs = []
epoch_losses = []
# loop over the dataset 100 times
for epoch in range(100):
 epoch_loss = 0.0
 # loop over batches
 for i, data in enumerate(data_loader):
  # retrieve inputs and labels for this batch
  x_batch, y_batch = data
  # zero the parameter gradients
  optimizer.zero_grad()
  # forward pass
  pred = model(x_batch)
  loss = criterion(pred, y_batch)
  # backward pass
  loss.backward()
  # SGD update
  optimizer.step()
  # update statistics
  epoch_loss += loss.item()
 # print error
 print(f'Epoch {epoch:5d}, loss {epoch_loss:.3f}')
 #send to plot
 epochs.append(epoch)
 epoch_losses.append(epoch_loss)
 # tell scheduler to consider updating learning rate
 scheduler.step()

In [ ]:
ax.plot(epochs, epoch_losses)
ax.set_xlabel('Epoch'); ax.set_ylabel('Loss')
ax.set_xlim([0,100]);ax.set_ylim([0,20])
ax.set_aspect(1.0)
plt.show()